<a href="https://colab.research.google.com/github/SchmetterlingIII/D.T./blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an improvement from the initial `spline-interp.ipynb` with a focus on a clearer preceding pseudocode as well as code structure, with classes and helper functions being developed first.

# Pseudocode

- Read in the data from the serial and unpack it so that it can be interpreted
- Clean up this data further such that the accelerometer data is cleaned (`madgwick-filter(accelerometer_data)`) and the output of this are the direction vectors of the tilt. *Further improvements to this function could also input the gyroscope and magnetometer data and have a more comprehensive input--the output will still be the direction vector of the tilt for each IMU so it doesn't change much*.
- Interpolate this appropriately (`forward-kinematics` then `cubic-spline-interp` is the current approach)
- Curvature analysis class that -- using whatever interpolated functions -- creates a list of the curvatures at each input
    - at first, the calibration phase happens where a distribution of the inputted curvatures (wrt their indices and time) are plotted for that duration (i.e. the state `calibration_duration`). This distribution is stored and then checked against for each instance outside of this test phase. I intend to use one-class support vector machine (SVM) but not sure currently. With enough tests (and ML that happens offline) I would like to produce a very inexpensive and accurate method of interpreting the data in the calibration phase but I am not sure currently and need guidance on how to interpret this data (in semi-real time).
    - After this, if an inputted curvature (i.e. instance of the digital twin of the user's spine) has deviated beyond a defined boundary (**how should this be defined?**) then the segment of the spine that this data falls within (which has a corresponding haptic motor) will be alerted through the serial and vibrate on the area of deviation. On the updated plot, there will be a red dots along the arclength of deviation for an update.
        - (for future proofing, I would also like for if there are high instances of static posture, but this feels like just another function and having the if condition be `if deviated or prolonged-static` where both of those are states within the class)
    - Otherwise, there is just a plot of the digital twin on the screen (what other metrics would be cool to have later?)
- plot the interpolated spline at a lower frequency than data is being interpreted and stored in database
- all of this data needs to be stored in a database for ML later and a clearer interpretation of what is happening at specific timesteps and to isolate errors that can be debugged

# Main

### Module Imports

In [ ]:
import serial.tools.list_ports
import string
import serial

import matplotlib.pyplot as plt
from  mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation

import time
from collections import deque
import numpy as np
from scipy.interpolate import CubicSpline
import scipy
import sys # specifically for debugging

### Serial Reading

In [ ]:
BAUDRATE = 115200
try:
    '''
    Basic setup for port communication
    '''

    ports = serial.tools.list_ports.comports()
    serialInst = serial.Serial()
    portList = [str(i) for i in ports]
    print(portList)

    com = input("Select COM PORT for Arduino: ")

    for i in range(len(portList)):
        if portList[i].startswith("COM" + str(com)):
            SERIAL_PORT = "COM" + str(com)
            print(SERIAL_PORT)

    serialInst.baudrate = BAUDRATE
    serialInst.port = SERIAL_PORT
    serialInst.open()
    print(f"Connected to {SERIAL_PORT} at {BAUDRATE} baud.")

    '''
    Initial data initialisation.
    This is a complementary function to the c++ code which says how many sensors are connected and which positions they are in.
    '''
    while True:
        line = serialInst.readline().decode('utf-8') # each line is the decoded form of the serial
        if line: # if there is data in the readline i.e if line == True
            print(f"Arduino: {line}")
        if "Available channels: " in line:
            channels_part = line.split(":")[-1].strip() # extract all data after colon
            # parse the csv into on stringed list
            IMU_ID_LIST = [id.strip() for id in channels_part.split(",") if id.strip()]
        if "Number of sensors: " in line:
           ID_NUM = int(line.strip(":")[-3]) # the number of read sensors, last instance is "\n" and so index = -3 is the appropriate index
           IMU_DEQUES = [deque(maxlen=50) for i in range(ID_NUM)]
        if "Waiting for 'begin program' command" in line:
            break

    '''
    Input of linear distances for the forward kinematics chain (and subsequent calculations)
    '''
    print("INSTRUCTIONS:\nInput the linear distances between your sensors in metres.\nMeasure from lowest to highest.\nI would recommend using a high resolution ruler to reduce drift.\n")
    linear_distances = []
    for i in range(ID_NUM - 1):
        value = float(input(f"{i + 1}: "))
        linear_distances.append(value)

    print("Sending 'begin' command to Arduino")
    serialInst.write(b'begin program') # sent in bytes rather than a high level string since it is sent to back to the compiler

    time.sleep(2)

### Helper Functions
- data filtering (madgwick filter)
- forward kinematics function
- cubic spline (outputting discretised function)

In [ ]:
def angle_tilt_filter(IMU_data):
    '''
    https://www.youtube.com/watch?v=7VW_XVbtu9k
    Use the above video to extract the angle between each of the IMUs.

    This is less susceptible to gyro tilt over time

    Return: normalised matrix containing the vector values of each of the filtered IMUs using this angle extraction method.
    '''
    return angle_filtered_data

def kalman_filter(angle_filtered_data):
    '''
    https://www.youtube.com/watch?v=5HuN9iL-zxU&list=PLeuMA6tJBPKsAfRfFuGrEljpBow5hPVD4&index=18
    Another tutorial that will help me actually have an accurate input of the data, without resorting to l'IA or learning the maths behind this.

    I will look into how Kalman filters work from a high level but if there is a python module for it then I will be happy to just copy it.
    '''
    return filtered_data

def forward_kinematics(filtered_data, linear_distances):
    '''
    Computes positions using the IMU data and distances, assuming that the base IMU is at the origin.
    Returns: list of 3D positions where the IMUs are in an arbitrary & scaled 3D space

    For later improvements, I will use quaternions to handle the tilt as done in the CHARM Lab device.
    '''

    # the data is in a matrix of the tilts of each of the sensors along the spine
    if not filtered_data:
        return None # will this return an error?

    origin = np.array([0,0,0])
    positions = [origin]
    cum_dist = 0

    for i in range(0, len(filtered_data)):
        # direction_vector is the vector needed to get from position_0 to position_1
        next_vector = np.array(filtered_data[i])
        current_vector = np.array(filtered_data[i-1])
        direction_vector = next_vector - current_vector

        norm = np.linalg.norm(direction_vector) # the unit direction of the vector

        if norm < 1e-10: # error handling
            # handle the error
        else:
            direction_vector = direction_vector / norm



